# Test du modéle

### Importation des bibliotéque

In [1]:
from sklearn.preprocessing import LabelEncoder 
import math
import cv2
import numpy as np
import pandas as pd
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
from IPython.display import HTML
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
import pickle

### Initialisation

In [2]:
# Initialisation de la classe mediapipe pose.
mp_pose = mp.solutions.pose

# Configuration de la fonction Pose.
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)

# Initialisation de la classe mediapipe drawing, utile pour l'annotation.
mp_drawing = mp.solutions.drawing_utils


### Chargement du modéle et définition des fonctions

In [3]:
# Charger le modèle à partir du fichier h5
model = load_model('CNN1.h5')

📏 Cette fonction, calculateAngle, calcule l'angle entre trois repères différents spécifiés en entrée et retourne cet angle en degrés.

In [4]:
def calculateAngle(landmark1, landmark2, landmark3):
    '''
    Cette fonction calcule l'angle entre trois landmarks différents.
    Args:
        landmark1: Le premier landmark contenant les coordonnées x, y et z.
        landmark2: Le deuxième landmark contenant les coordonnées x, y et z.
        landmark3: Le troisième landmark contenant les coordonnées x, y et z.
    Returns:
        angle: L'angle calculé entre les trois landmarks.
    '''

    # Obtenir les coordonnées des landmarks requis.
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    # Calculer l'angle entre les trois points.
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    # Vérifier si l'angle est inférieur à zéro.
    if angle < 0:

        # Ajouter 360 à l'angle trouvé.
        angle += 360
    
    # Renvoyer l'angle calculé.
    return angle



🕺 Cette fonction, nommée detectPose, a pour objectif de détecter et d'analyser la pose humaine dans une image donnée. Elle prend en entrée une image, un objet de détection de pose, et un paramètre optionnel pour l'affichage. La fonction effectue la détection de la pose en utilisant l'objet de pose fourni, récupère les points de repère détectés, puis calcule une série d'angles pour différentes articulations du corps, tels que les angles des coudes, des épaules, des hanches, des poignets, etc. Si le paramètre d'affichage est activé, la fonction affiche l'image d'origine avec les annotations de la pose. En fin de compte, la fonction retourne l'image de sortie, la liste des points de repère détectés, ainsi que les angles calculés pour les articulations du corps. 📸

In [5]:
def detectPose(image, pose, display=True):
    '''
    Cette fonction effectue la détection de la pose sur une image.
    Args:
        image: L'image d'entrée avec une personne en évidence dont les repères de pose doivent être détectés.
        pose: La fonction de configuration de la pose requise pour effectuer la détection de la pose.
        afficher: Une valeur booléenne qui, si elle est définie sur True, affiche l'image d'entrée d'origine, l'image résultante,
                 et les repères de pose dans le tracé 3D, et ne renvoie rien.
    Returns:
        image_resultat: L'image d'entrée avec les repères de pose détectés dessinés.
        repères: Une liste de repères détectés convertis à leur échelle d'origine.
    '''
    
    # Cette fonction effectue la détection de la pose sur une image.
    output_image = image.copy()
    
    # Convertissez l'image de BGR en format RGB.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Effectuer la détection de la pose.
    results = pose.process(imageRGB)
    
    # Récupérer la hauteur et la largeur de l'image d'entrée.
    height, width, _ = image.shape
    
    # Initialisez une liste pour stocker les repères détectés.
    landmarks = []
    
    # Vérifiez si des repères sont détectés.
    if results.pose_landmarks:
    
        # Dessinez les repères de pose sur l'image de sortie.
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS)
        
        # Itérer sur les repères détectés.
        for landmark in results.pose_landmarks.landmark:
            
            # Ajoutez le repère à la liste.
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                                  (landmark.z * width)))
    
    # Vérifiez si l'image d'entrée d'origine et l'image résultante doivent être affichées.
    if display:
    
        # Affichez l'image d'entrée d'origine et l'image résultante.
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
        # Tracez également les repères de pose en 3D.
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
        
   
    else:
        
        # renvoyez l'image de sortie et les repères trouvés.
        return output_image, landmarks

🧘 La fonction classifyPose a pour objectif de classifier les poses de boxe chinoise en fonction des angles des différentes articulations du corps détectés. Elle prend en entrée une liste de points de repère de la pose 📍, une image de sortie avec les points de repère dessinés 🖼️, un modèle de classification 🤖, une liste de catégories de poses 📋, ainsi qu'un paramètre optionnel de visualisation 👁️.

🔄 Initialement, la fonction initialise l'étiquette de la pose à "Unknown Pose" (pose inconnue) ❓ et définit la couleur pour écrire l'étiquette sur l'image (rouge 🔴).

📏 Ensuite, elle calcule une série d'angles entre les articulations du corps, tels que les angles des coudes, des épaules, des hanches, des poignets, etc. Ces angles sont calculés à partir des points de repère détectés dans la pose.

📊 La fonction prépare ensuite les données des angles calculés dans un format adapté pour la classification, puis prédit la pose à l'aide du modèle de classification.

🎯 Si la prédiction dépasse un certain seuil de confiance (0,95 dans cet exemple), l'étiquette de la pose est mise à jour en fonction de la prédiction du modèle.

🖋️ Enfin, la fonction écrit l'étiquette de la pose sur l'image de sortie, en utilisant la couleur appropriée en fonction du succès de la classification.

🖼️ Lorsque le paramètre display est défini sur True, l'image résultante est affichée. Sinon, l'image de sortie et l'étiquette de la pose classifiée sont renvoyées en sortie de la fonction.

In [6]:
def classifyPose(landmarks, output_image, model,categories, display=False):
    '''
    Cette fonction classifie les poses de yoga en fonction des angles des différentes articulations du corps.
    Args:
        landmarks: Une liste de repères détectés de la personne dont la pose doit être classifiée.
        output_image: Une image de la personne avec les repères de pose détectés dessinés.
        afficher: Une valeur booléenne qui, si elle est définie sur True, affiche l'image résultante avec l'étiquette de pose
                 écrite dessus et ne renvoie rien.
    Returns:
        output_image: L'image avec les repères de pose détectés dessinés et l'étiquette de pose écrite.
        label: L'étiquette de pose classifiée de la personne dans l'image de sortie.
    '''
    
    # Initialiser l'étiquette de la pose. Elle n'est pas connue à ce stade.
    label = 'Unknown Pose'

    # Spécifier la couleur (rouge) avec laquelle l'étiquette sera écrite sur l'image.
    color = (0, 0, 255)
    
    # Calculer les angles nécessaires.
    #----------------------------------------------------------------------------------------------------------------
    
# Calculer les angles articulaires pour chaque ligne dans le DataFrame.

    # Calculer les angles des articulations
    # Obtenir l'angle entre les points d'épaule gauche, de coude gauche et de poignet gauche.
    elbow_left_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])

    # Obtenir l'angle entre les points d'épaule droite, de coude droit et de poignet droit.
    elbow_right_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])

    # Obtenir l'angle entre les points de hanche gauche, d'épaule gauche et de coude gauche.
    shoulder_left_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value])

    # Obtenir l'angle entre les points de hanche droite, d'épaule droite et de coude droit.
    shoulder_right_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
    

    # Obtenir l'angle entre les points de hanche gauche, de genou gauche et de cheville gauche.
    hip_left_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value])

    # Obtenir l'angle entre les points de hanche droite, de genou droit et de cheville droite.
    hip_right_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])

    # Obtenir l'angle entre les points de coude gauche, de poignet gauche et de petit doigt gauche.
    wrist_left_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value])

    # Obtenir l'angle entre les points de coude droit, de poignet droit et de petit doigt droit.
    wrist_right_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value])
    
    # Obtenir l'angle entre les points de hanche droit, de épole gauche et de coude gauche.
    shoulder_lateral_left_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value])
    
    # Obtenir l'angle entre les points de hanche gauche, de épole droit et de coude droit.
    shoulder_lateral_right_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
    
     # Obtenir l'angle entre les points de hanche droit, de épole gauche et de coude gauche.
    shoulder_lateral2_left_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value])
    
    # Obtenir l'angle entre les points de hanche gauche, de épole droit et de coude droit.
    shoulder_lateral2_right_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
    
    
    # Obtenir l'angle entre les points de pouce gauche, de poignet gauche et de petit doigt gauche.
    hand_left_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value],
                                       landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value],
                                       landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value])
    
    # Obtenir l'angle entre les points de pouce droit, de poignet droit et de petit doigt droit.
    hand_right_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value])
    
    # Obtenir l'angle entre les points de pouce gauche, de poignet gauche et de coude gauche.
    hand_lateral_left_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value],
                                       landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value],
                                       landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value])
    
    # Obtenir l'angle entre les points de pouce droit, de poignet droit et de coude droit.
    hand_lateral_right_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
    
    
    #----------------------------------------------------------------------------------------------------------------

    
    dic = {'Angles_Elbow_Right':[elbow_right_angle],
                        'Angles_Elbow_Left': [elbow_left_angle],
                        'Angles_Shoulder_Right': [shoulder_right_angle],
                        'Angles_Shoulder_Left': [shoulder_left_angle],
                        'Angles_Hip_Right': [hip_right_angle],
                        'Angles_Hip_Left': [hip_left_angle],
                        'Angles_Wrist_Right': [wrist_right_angle],
                        'Angles_Wrist_Left':[wrist_left_angle],
                        'Angles_Shoulder_lateral_Left': [shoulder_lateral_left_angle],
                        'Angles_Shoulder_lateral_right': [shoulder_lateral_right_angle],
                        'Angles_Shoulder_lateral2_Left': [shoulder_lateral2_left_angle],
                        'Angles_Shoulder_lateral2_right': [shoulder_lateral2_right_angle],
                        'Angles_Hand_Left': [hand_left_angle],
                        'Angles_Hand_right': [hand_right_angle],
                        'Angles_Hand_lateral_Left': [hand_lateral_left_angle],
                        'Angles_Hand_lateral_right': [hand_lateral_right_angle]}
    
    new_data = pd.DataFrame(dic)
        #---------------------------------------------------------------------------------------------------------------


    # Prédire de nouvelles positions
    predictions = model.predict(new_data)
    
    best_prediction_index = np.argmax(predictions)
    best_prediction_label = categories[best_prediction_index]
    best_prediction_value = np.max(predictions)

    if best_prediction_value > 0.95:
        label = best_prediction_label
        
    #----------------------------------------------------------------------------------------------------------------

    # Vérifier si la pose est classifiée avec succès
    if label != 'Unknown Pose':

        # Mettre à jour la couleur (en vert) avec laquelle l'étiquette sera écrite sur l'image.
        color = (0,255,0)  

    # Écrire l'étiquette sur l'image de sortie. 
    cv2.putText(output_image, label, (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, color, 5)

    # Vérifier si l'image résultante doit être affichée.
    if display:

        # Afficher l'image résultante..
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');

    else:

        # Renvoyer l'image de sortie et l'étiquette classifiée.
        return output_image, label

🧾 Ce code charge un LabelEncoder à partir d'un fichier (dans ce cas, 'label_encoder.pkl') à l'aide du module pickle 📦. Un LabelEncoder est couramment utilisé pour convertir des étiquettes de texte en étiquettes numériques pour être utilisé dans des modèles de machine learning 🤖.

📥 Après avoir chargé le LabelEncoder, le code obtient la liste des noms de catégories à partir des classes du LabelEncoder 📃. Ces noms de catégories sont généralement des étiquettes textuelles qui correspondent aux classes numériques 📊.

In [7]:
# Charger le LabelEncoder à partir du fichier
with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)
    
# Obtention de la liste des noms de catégories
categories = label_encoder.classes_.tolist()

# Affichage des noms de catégories
print(categories)

['Bong_Sau_L', 'Bong_Sau_R', 'Double_Biu_Sau', 'Double_Bui_Sau', 'Double_Fak_Sau', 'Double_Forward_Palm', 'Double_Forward_Palm_prep', 'Double_Gan_Sau', 'Double_Gum_Sau', 'Double_Huan_Sau', 'Double_Huen_Sau', 'Double_Jam_Sau', 'Double_Jut_Sau', 'Double_Jut_Sau_0', 'Double_Jut_Sau_1', 'Double_Kwan_Sau', 'Double_Lan_Sau_L', 'Double_Lan_Sau_R', 'Double_Rear_Palm', 'Double_Rear_Palm/Double_Forward_Palm', 'Double_Tai_Sau', 'Double_Tan_Sau', 'Double_Tok_Sau', 'Double_Tok_Sau_0', 'Double_Tok_Sau_1', 'Down_Low_Palm_L', 'Down_Low_Palm_R', 'Fook_Sau_L', 'Fook_Sau_R', 'Gan_Sau_L', 'Gum_Sau', 'Gum_Sau_L', 'Gum_Sau_L+Gum_Sau_R', 'Gwat_Sau_L', 'Gwat_Sau_L_m', 'Gwat_Sau_R', 'Gwat_Sau_R_m', 'Hight_Low_Palm_L', 'Hight_Low_Palm_R', 'Huen_Sau_L', 'Huen_Sau_R', 'Jam_Sau_L', 'Jam_Sau_R', 'Lau_Sao_L', 'Lau_Sao_R', 'Low_Sau_L', 'Low_Sau_R', 'Low_Side_Palm_L', 'Low_Side_Palm_R', 'Make_Fist_L', 'Make_Fist_R', 'Pak_Sau_L', 'Pak_Sau_R', 'Palm Strike', 'Palm_Strike_L', 'Palm_Strike_R', 'Punch_L', 'Punch_R', 'Side_

# WEBCAM

📹 Ce code utilise la bibliothèque OpenCV pour capturer la vidéo en temps réel à partir de la webcam 🎥 et classer la pose d'une personne. Voici comment il fonctionne :

🔄 Une fonction detectAndClassifyPose est définie pour détecter et classifier la pose dans chaque image capturée depuis la webcam. La fonction prend en entrée l'image capturée (frame), l'objet de détection de pose (pose), et les modèles de classification (model) et les catégories (categories) pour classer la pose.

📷 La boucle principale commence en utilisant cv2.VideoCapture(0) pour capturer la vidéo à partir de la webcam de l'ordinateur. La caméra est identifiée par l'indice 0, ce qui signifie la première caméra détectée.

🔄 À chaque itération de la boucle, une image est capturée à partir de la webcam à l'aide de cap.read(), et la fonction detectAndClassifyPose est appelée pour détecter et classifier la pose dans cette image.

🎯 La classification de la pose est réalisée en utilisant le modèle de classification (model) et les catégories (categories) chargés précédemment.

🖥️ L'image avec la pose classifiée est affichée dans une fenêtre nommée "Pose Classification" à l'aide de cv2.imshow.

🛑 La boucle continue à s'exécuter jusqu'à ce que la touche 'q' soit pressée. Lorsque 'q' est pressé, la boucle se termine, et les ressources de la webcam sont libérées avec cap.release() et toutes les fenêtres OpenCV sont fermées avec cv2.destroyAllWindows().

In [8]:
import cv2

# Fonction pour détecter et classifier la pose
def detectAndClassifyPose(frame, pose):
    output_image, landmarks = detectPose(frame, pose, display=False)
    if landmarks:
        classifyPose(landmarks, output_image, model, categories, display=False)
    cv2.imshow("Pose Classification", output_image)

# Capture vidéo à partir de la webcam
cap = cv2.VideoCapture(0)

# Boucle principale pour lire les images de la webcam
while True:
    ret, frame = cap.read()

    # Appeler la fonction pour détecter et classifier la pose sur chaque image
    detectAndClassifyPose(frame, pose)

    # Sortir de la boucle si la touche 'q' est pressée
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 18ms/step


# Video

In [9]:
import cv2

# Fonction pour détecter et classifier la pose
def detectAndClassifyPose(frame, pose):
    output_image, landmarks = detectPose(frame, pose, display=False)
    if landmarks:
        classifyPose(landmarks, output_image, model, categories, display=False)
    cv2.imshow("Pose Classification", output_image)

# Capture vidéo à partir de la webcam
cap = cv2.VideoCapture('../video/Siu Nim Tau/test/Siu Nim Tau Summary (詠春小念頭).mp4')

# Boucle principale pour lire les images de la webcam
while True:
    ret, frame = cap.read()

    # Appeler la fonction pour détecter et classifier la pose sur chaque image
    detectAndClassifyPose(frame, pose)

    # Sortir de la boucle si la touche 'q' est pressée
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 17ms/step
